In [1]:
from music21 import converter, instrument, note, chord
import numpy as np
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dense, Conv1D, Dropout, GlobalMaxPool1D
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import pickle

Using plaidml.keras.backend backend.


In [2]:
def read_midi(file_path): 
    notes = [] 
    notes_to_parse = None 


    midi= converter.parse(file_path)
    
    s2 = instrument.partitionByInstrument(midi)
    for part in s2.parts:
    
        #select elements of only piano
        if 'Piano' in str(part): 
        
            notes_to_parse = part.recurse() 
      
            #finding whether a particular element is note or a chord
            for element in notes_to_parse:
                
                #note
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                
                #chord
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

    return np.array(notes)


base_dir = '../50cent'
cent_files = [f'{base_dir}/{i}' for i in os.listdir(base_dir)]
cent_notes = [read_midi(i) for i in tqdm(cent_files)]
all_notes = [i for note_ in cent_notes for i in note_]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:12<00:00,  1.10it/s]


In [3]:
from collections import defaultdict 

note_counter = defaultdict(int)

for i in all_notes: 
    note_counter[i]+= 1

In [ ]:
int_to_note = dict((idx, n) for idx, n in enumerate(all_notes))

int_to_note

In [ ]:
def get_ts(notes, timestep = 10): 
    x = [] 
    y = [] 

    pbar = tqdm(notes)
    for idx_song, song in enumerate(pbar):
        for idx in range(timestep,len(song)-(timestep), 1): 
            x.append(song[idx:idx+timestep])
            y.append(song[idx + (timestep)])

    x = np.array(x)
    y = np.array(y)
    
    return x, y

x,y = get_ts(cent_notes)

In [ ]:
def unique_set(np_array, array_type): 
    unique = list(set(np_array.ravel()))
    unique_int = dict((n,i) for i,n in enumerate(unique))
    new_array =[] 
    if array_type == 'x':
        for group in np_array: 
            temp = [] 
            for n in group: 
                temp.append(unique_int[n])
            new_array.append(temp)
    else: 
        for n in np_array: 
            new_array.append(unique_int[n])
    new_array = np.array(new_array)
    return new_array, unique_int

new_x, x_dict = unique_set(x, array_type = 'x')
new_y, y_dict = unique_set(y, array_type ='y')

new_x,new_y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, train_size = .90, random_state = 10)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)

len(x_train), len(x_test)

In [ ]:
assert False

In [ ]:
def get_lstm(n): 
    model = Sequential()
    model.add(LSTM(128, return_sequences = True))
    model.add(LSTM(128))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dense(n, activation = 'softmax')) 
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam')
    return model

In [ ]:
# unique_n = len(set(new_y))
# get_lstm(unique_n, x_train)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/LSTM.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')



batch = 128 
epochs = 1000 
callbacks = [early_stopping, model_checkpoint]

unique_n = len(set(new_y))
lstm_model = get_lstm(unique_n)

lstm_history = lstm_model.fit(x_train, y_train, batch_size = batch, epochs = epochs, validation_data = (x_test, y_test), 
                        callbacks = callbacks)


In [ ]:
lstm_model = load_model('ModelWeights/LSTM.h5')
print(lstm_model.summary())

In [ ]:
import random 

idx = np.random.randint(0, len(x_test)-1)
random_music = x_test[idx]
predictions = [] 
for i in range(10): 
    random_music = random_music.reshape(1,timestep,1)
    prob = lstm_model.predict(random_music)[0]
    y_pred = np.argmax(prob, axis = 0)
    
    predictions.append(y_pred)
    
    random_music = np.insert(random_music[0], len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

In [ ]:
unique_x = list(set(x.ravel()))

x_int_dict = dict((i,n) for n,i in enumerate(unique_x))

predicted_notes = [x_int_dict[i] for i in predictions]
predicted_notes